In [2]:
import mne
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import json
from torch import tensor
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import label_binarize

mne.set_log_level('ERROR')

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [4]:
#sike we doing transformers cuz my laptop is shit
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class EEGTransformer(nn.Module):
    def __init__(self, input_dim, num_channels, num_classes, d_model=64, nhead=8, num_encoder_layers=2, dropout=0.1):
        super(EEGTransformer, self).__init__()

        # Temporal Transformer Encoder Setup Type Shit
        temporal_encoder_layer = TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
        self.temporal_encoder = TransformerEncoder(temporal_encoder_layer, num_layers=num_encoder_layers)

        # Spatial Transformer Encoder Setup Sligthly less type
        spatial_encoder_layer = TransformerEncoderLayer(d_model=num_channels, nhead=nhead, dropout=dropout)
        self.spatial_encoder = TransformerEncoder(spatial_encoder_layer, num_layers=num_encoder_layers)

        self.input_fc = nn.Linear(input_dim, d_model)
        self.output_fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        # x shape: (batch_size, num_channels, sequence_length)

        # Process each channel with the temporal encoder
        # Transpose x to match the input shape expected by TransformerEncoder (L, N, E)
        x = x.permute(2, 0, 1)
        x = self.input_fc(x)  
        x = self.temporal_encoder(x)

        # Transpose back to (batch_size, d_model, sequence_length) and apply spatial encoder
        x = x.permute(1, 2, 0)  
        x = self.spatial_encoder(x)

        
        x = torch.mean(x, dim=2)  
        x = self.output_fc(x)
        return F.log_softmax(x, dim=1)

# Parameters
input_dim = 256  
num_channels = 23  
num_classes = 3 

model = EEGTransformer(input_dim=input_dim, num_channels=num_channels, num_classes=num_classes)


AssertionError: embed_dim must be divisible by num_heads